# **_Neighbourhoods in Toronto Project_**

### Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [4]:
#Packages
#--Web scraping packages
from bs4 import BeautifulSoup
import requests
#Pandas/numpy for data manipulation
import pandas as pd
import numpy as np

In [5]:
#load URL we want to scrape into an array
BASE_URL = [
'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
]

In [6]:
#loading empty array for postal codes
postal_codes = []
#Loop through our URLs we loaded above
for b in BASE_URL:
    html = requests.get(b).text
    soup = BeautifulSoup(html, "html.parser")
#identify table we want to scrape
postalcodes_table = soup.find('table', {"class" : "wikitable sortable"})
#print(postalcodes_table)
#try clause to skip any companies with missing/empty board member tables
try:
#loop through table, grab each of the 4 columns shown (try one of the links yourself to see the layout)
    for row in postalcodes_table.find_all('tr'):
        cols = row.find_all('td')
        #print(len(cols))
        if len(cols) == 3:
            postal_codes.append((b, cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))
            #print(postal_codes)
except: pass  

In [7]:
#convert output to new array, check length
postalcodes_array = np.asarray(postal_codes)
#len(postalcodes_array)
#print(postalcodes_array)

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [8]:
#convert new array to dataframe
df = pd.DataFrame(postalcodes_array)
df.drop(0, axis=1, inplace=True, errors='raise')
#rename columns, check output
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [9]:
# Get names of indexes for which column Borough == "Not assigned"
indexNames = df[df.Borough == "Not assigned"].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True, errors='raise')
# set 'Neighbourhood' to 'Borough' when 'condition_column' is "Not assigned"
df.loc[df['Neighbourhood'] == "Not assigned", 'Neighbourhood'] = df.Borough
df.iloc[0:10]

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [10]:
#df.groupby(by='Postcode')['Neighbourhood'].apply(','.join).reset_index()
#foo = lambda a: "/".join(a)
#df.groupby(by='Postcode').agg({'Neighbourhood': foo}).reset_index()
#df.groupby('Postcode').agg({'Neighbourhood': foo}).reset_index()

a = 2
for i in df.index: 
    #print(df['Postcode'][i], df['Neighbourhood'][i], df['Postcode'][a]) 
    #print(df.index[i], i, a)  
    if ((df['Postcode'][i] == df['Postcode'][a]) & (a != 2)):
        #print('Yes', i, a)
        df['Neighbourhood'][i] = df['Neighbourhood'][i] +', '+ df['Neighbourhood'][a]
        df.drop(a, inplace=True, errors='raise')  
        #df['Neighbourhood'][i] = df['Neighbourhood'][i].str.cat(df['Neighbourhood'][a],sep=", ")  
    a = i
df   

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
7,M6A,North York,"Lawrence Manor, Lawrence Heights"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,"Malvern, Rouge"
14,M3B,North York,Don Mills North
16,M4B,East York,"Parkview Hill, Woodbine Gardens"
18,M5B,Downtown Toronto,"Garden District, Ryerson"


### Use the .shape method to print the number of rows of your dataframe.

In [12]:
df.shape

(103, 3)